<a href="https://colab.research.google.com/github/aloy4646/football_video_classification/blob/main/evaluate_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import tensorflow as tf
import random
import pathlib
import cv2
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

## Evaluasi model

### Load test data

In [ ]:
HEIGHT = 224
WIDTH = 224

In [ ]:
class FrameGeneration:
  def __init__(self, path, n_frames, training = False):
    """ Mengembalikan set frames beserta label terkait.

      Args:
        path: Path file video.
        n_frames: Jumlah frames.
        training: Boolean untuk menentukan apakah dataset training sedang dibuat.
    """
    self.path = path
    self.n_frames = n_frames
    self.training = training
    self.class_names = sorted(set(p.name for p in self.path.iterdir() if p.is_dir()))
    self.class_ids_for_name = dict((name, idx) for idx, name in enumerate(self.class_names))

  def get_files_and_class_names(self):
    # video_paths = list(self.path.glob('*/*.avi'))
    video_paths = list(self.path.glob('*/*.mp4'))
    classes = [p.parent.name for p in video_paths]
    return video_paths, classes

  def frames_from_video_file(self, video_path, n_frames, output_size = (HEIGHT,WIDTH)):
    """
      Membuat frames dari setiap file video yang ada untuk setiap kategori.

      Args:
        video_path: Path file video.
        n_frames: Jumlah frames yang akan dibuat per file video.
        output_size: Ukuran piksel dari frame keluaran.

      Return:
        Array NumPy dari frames dengan bentuk (n_frames, HEIGHT, WIDTH, saluran).
    """
    # Read each video frame by frame
    result = []
    src = cv2.VideoCapture(str(video_path))

    video_length = src.get(cv2.CAP_PROP_FRAME_COUNT)

    # Frame step fleksibel mengikuti panjang dari sumber video
    frame_step = max(int(video_length / (n_frames-1)), 1)

    need_length = 1 + (n_frames-1) * frame_step

    if need_length > video_length:
      start = 0
    else:
      max_start = video_length - need_length
      start = random.randint(0, max_start + 1)

    src.set(cv2.CAP_PROP_POS_FRAMES, start)

    # ret is a boolean indicating whether read was successful, frame is the image itself
    ret, frame = src.read()
    result.append(self.format_frames(frame, output_size))

    for _ in range(n_frames - 1):
      for _ in range(frame_step):
        ret, frame = src.read()
      if ret and frame is not None:
        frame = self.format_frames(frame, output_size)
        result.append(frame)
      else:
        result.append(np.zeros_like(result[0]))

    src.release()
    result = np.array(result)[..., [2, 1, 0]]
    return result


  def format_frames(self, frame, output_size):
      """
      Menambahkan padding dan meresize frame dari video.

      Args:
          frame: frame yang perlu diresize dan dipad.
          output_size: Ukuran piksel dari frame keluaran.

          Return:
            Frame yang diformat dengan padding sesuai ukuran keluaran.
      """
      # return resized_frame
      frame = tf.image.convert_image_dtype(frame, tf.float32)
      frame = tf.image.resize_with_pad(frame, *output_size)
      return frame

  def __call__(self):
    video_paths, classes = self.get_files_and_class_names()

    pairs = list(zip(video_paths, classes))

    if self.training:
      random.shuffle(pairs)

    for path, name in pairs:
      # frame retrieval
      video_frames = self.frames_from_video_file(path, self.n_frames)
      label = self.class_ids_for_name[name] # Encode labels
      yield video_frames, label, path.name

In [ ]:
dataset_dir = pathlib.Path('/content/drive/MyDrive/AnwarSukoco/dataset_final/')

n_frames = 10

In [ ]:
batch_size = 1

# (n_frames, HEIGHT, WIDTH)
output_signature = (tf.TensorSpec(shape = (None, None, None, 3), dtype = tf.float32),
                    tf.TensorSpec(shape = (), dtype = tf.int16),
                    tf.TensorSpec(shape = (), dtype = tf.string))

test_ds = tf.data.Dataset.from_generator(FrameGeneration(dataset_dir / 'test', n_frames),
                                         output_signature = output_signature)

test_ds = test_ds.batch(batch_size)

### Evaluate

In [ ]:
class Evaluate:
  def plot_confusion_matrix(actual, predicted, labels, konfiguration_number, path):
    cm = tf.math.confusion_matrix(actual, predicted)
    ax = sns.heatmap(cm, annot=True, fmt='g')
    sns.set(rc={'figure.figsize':(12, 12)})
    sns.set(font_scale=1.4)
    ax.set_title('Confusion matrix of model no. ' + str(konfiguration_number))
    ax.set_xlabel('Predicted Action')
    ax.set_ylabel('Actual Action')
    plt.xticks(rotation=90)
    plt.yticks(rotation=0)
    ax.xaxis.set_ticklabels(labels)
    ax.yaxis.set_ticklabels(labels)
    plt.savefig(path, bbox_inches='tight')
    plt.close()
    plt.clf()

  def get_actual_predicted_labels(dataset, model):
    """
      Membuat daftar nilai ground truth aktual dan prediksi dari model.

      Args:
        dataset: Struktur data yang dapat diiterasi, seperti TensorFlow Dataset, dengan fitur dan label.

      Return:
        Nilai ground truth dan prediksi untuk dataset tertentu.
    """

    actual = []
    confidence = []
    predicted = []
    filenames = []

    for features, labels, filename in dataset.unbatch():
        actual.append(labels)

        # Features adalah 10 frame yang dianggap menjadi 1 video
        reshaped_features = tf.expand_dims(features, axis=0)
        prediction = model.predict(reshaped_features, verbose=0)
        predicted_classes = tf.argmax(prediction, axis=1).numpy()[0]
        predicted.append(predicted_classes)
        confidence.append(prediction[0])
        filenames.append(filename.numpy().decode())

    actual = tf.stack(actual, axis=0)
    predicted = tf.stack(predicted, axis=0)
    confidence = tf.constant(confidence)

    return actual, confidence, predicted, filenames

  def calculate_classification_metrics(y_actual, y_pred, labels):
    """
    Menghitung precision, recall, dan F1 score dari model klasifikasi menggunakan nilai ground truth dan
    prediksi.

    Args:
      y_actual: Label ground truth.
      y_pred: Label prediksi.
      labels: Daftar label klasifikasi.

    Return:
      Ukuran precision, recall, dan F1 score.
    """
    cm = tf.math.confusion_matrix(y_actual, y_pred)
    tp = np.diag(cm)  # Diagonal mewakili true positives
    precision = dict()
    recall = dict()
    f1_score = dict()

    for i in range(len(labels)):
        col = cm[:, i]
        row = cm[i, :]

        fp = np.sum(col) - tp[i]  # Jumlah kolom minus true positive adalah false positif
        fn = np.sum(row) - tp[i]   # Jumlah baris minus true positive adalah false negative

        precision[labels[i]] = tp[i] / (tp[i] + fp) if (tp[i] + fp) != 0 else 0  # Precision
        recall[labels[i]] = tp[i] / (tp[i] + fn) if (tp[i] + fn) != 0 else 0  # Recall

        # Calculate F1 score
        f1_score[labels[i]] = 2 * (precision[labels[i]] * recall[labels[i]]) / \
                              (precision[labels[i]] + recall[labels[i]]) if \
                              (precision[labels[i]] + recall[labels[i]]) != 0 else 0

    accuracy = np.sum(tp) / np.sum(cm)  # Accuracy

    mean_precision = np.mean(list(precision.values()))
    mean_recall = np.mean(list(recall.values()))
    mean_f1_score = np.mean(list(f1_score.values()))

    return accuracy, precision, recall, f1_score, mean_precision, mean_recall, mean_f1_score

  def simpan_hasil_prediksi_per_video(actual, confidence, predicted, filename, nomor_model):
    arr_actual = actual.numpy()
    arr_confidence = confidence.numpy()
    arr_predicted = predicted.numpy()

    csv_hasil_prediksi = []

    for i in range(len(filename)):
      formatted_confidence = ["{:.10f}".format(val) for val in arr_confidence[i]]
      csv_hasil_prediksi.append({
          "Filename": filename[i],
          "Actual":arr_actual[i],
          "Predicted":arr_predicted[i],
          "Confidence[0]":formatted_confidence[0],
          "Confidence[1]":formatted_confidence[1],
          "Confidence[2]":formatted_confidence[2],
          "Confidence[3]":formatted_confidence[3],
          "Confidence[4]":formatted_confidence[4],
          "Confidence[5]":formatted_confidence[5]
      })

      df_hasil_prediksi = pd.DataFrame(csv_hasil_prediksi)

      csv_path = "/content/drive/MyDrive/AnwarSukoco/csv_pengujian/pengujian_per_file/model_" + str(nomor_model) + ".csv"
      df_hasil_prediksi.to_csv(csv_path, index=False)

In [ ]:
fg = FrameGeneration(dataset_dir / 'test', n_frames)
labels = list(fg.class_ids_for_name.keys())

In [ ]:
directory = "/content/drive/MyDrive/AnwarSukoco/hasil_pelatihan"
directory_gambar_confusion_matrix = "/content/drive/MyDrive/AnwarSukoco/gambar_confusion_matrix"
csv_akurasi = []
csv_precision = []
csv_recall = []
csv_f1_score = []

for konfigurasi_folder in sorted(os.listdir(directory)):
    konfigurasi_path = os.path.join(directory, konfigurasi_folder)

    # Periksa apakah itu adalah folder
    if os.path.isdir(konfigurasi_path):
        print("==========================================")
        print(f"Mengolah folder konfigurasi: {konfigurasi_folder}")
        model_path = os.path.join(konfigurasi_path, "model")

        for checkpoint_folder in sorted(os.listdir(model_path)):
          print(f"---Mengolah folder checkpoint: {checkpoint_folder}")
          checkpoint_path = os.path.join(model_path, checkpoint_folder)

          # load dan menguji model
          model = tf.keras.models.load_model(checkpoint_path)

          actual, confidence, predicted, filenames = Evaluate.get_actual_predicted_labels(test_ds, model)

          # plot confusion matrix setiap model
          plt.clf()
          path_confusion_matrix = os.path.join(directory_gambar_confusion_matrix, checkpoint_folder)
          # mengambil nomor model
          nomor_model = ''.join(filter(str.isdigit, checkpoint_folder))
          Evaluate.plot_confusion_matrix(actual, predicted, labels, int(nomor_model), path_confusion_matrix)

          Evaluate.simpan_hasil_prediksi_per_video(actual, confidence, predicted, filenames, nomor_model)

          # # menghitung precision, recall, dan f1_score
          accuracy, precision, recall, f1_score, mean_precision, mean_recall, mean_f1_score = Evaluate.calculate_classification_metrics(actual,
                                                                                                                                        predicted,
                                                                                                                                        labels)

          csv_akurasi.append({
              "Nomor model": nomor_model,
              "Akurasi": accuracy
          })

          csv_precision.append({
              "Nomor model": nomor_model,
              "Card": precision['card'],
              "Celebration": precision['celebration'],
              "Corner": precision['corner'],
              "Foul": precision['foul'],
              "Shot": precision['shot'],
              "Substitution": precision['substitution'],
              "Average": mean_precision
          })

          csv_recall.append({
              "Nomor model": nomor_model,
              "Card": recall['card'],
              "Celebration": recall['celebration'],
              "Corner": recall['corner'],
              "Foul": recall['foul'],
              "Shot": recall['shot'],
              "Substitution": recall['substitution'],
              "Average": mean_recall
          })

          csv_f1_score.append({
              "Nomor model": nomor_model,
              "Card": f1_score['card'],
              "Celebration": f1_score['celebration'],
              "Corner": f1_score['corner'],
              "Foul": f1_score['foul'],
              "Shot": f1_score['shot'],
              "Substitution": f1_score['substitution'],
              "Average": mean_f1_score
          })

          print("------Accuracy: " + str(accuracy))
          print("------Precision: " + str(precision))
          print("------Mean Precision: " + str(mean_precision))
          print("------Recall: " + str(recall))
          print("------Mean Recall: " + str(mean_recall))
          print("------f1 score: " + str(f1_score))
          print("------Mean f1 score: " + str(mean_f1_score))

df_akurasi = pd.DataFrame(csv_akurasi)
df_precision = pd.DataFrame(csv_precision)
df_recall = pd.DataFrame(csv_recall)
df_f1_score = pd.DataFrame(csv_f1_score)

# Menyimpan DataFrame sebagai file CSV
csv_path = "/content/drive/MyDrive/AnwarSukoco/csv_pengujian/"
df_akurasi.to_csv(csv_path+"akurasi.csv", index=False)
df_precision.to_csv(csv_path+"precision.csv", index=False)
df_recall.to_csv(csv_path+"recall.csv", index=False)
df_f1_score.to_csv(csv_path+"f1_score.csv", index=False)

Mengolah folder konfigurasi: konfigurasi_1-4-7_10
---Mengolah folder checkpoint: checkpoint_1
------Accuracy: 0.5666666666666667
------Precision: {'card': 0.0, 'celebration': 0, 'corner': 0.4166666666666667, 'foul': 0.8333333333333334, 'shot': 1.0, 'substitution': 1.0}
------Mean Precision: 0.5416666666666666
------Recall: {'card': 0.0, 'celebration': 0.0, 'corner': 1.0, 'foul': 1.0, 'shot': 1.0, 'substitution': 0.4}
------Mean Recall: 0.5666666666666667
------f1 score: {'card': 0, 'celebration': 0, 'corner': 0.5882352941176471, 'foul': 0.9090909090909091, 'shot': 1.0, 'substitution': 0.5714285714285715}
------Mean f1 score: 0.511459129106188
---Mengolah folder checkpoint: checkpoint_10
------Accuracy: 0.6666666666666666
------Precision: {'card': 0.8, 'celebration': 0.0, 'corner': 0.8333333333333334, 'foul': 0.7142857142857143, 'shot': 1.0, 'substitution': 0.4}
------Mean Precision: 0.6246031746031746
------Recall: {'card': 0.8, 'celebration': 0.0, 'corner': 1.0, 'foul': 1.0, 'shot': 0

<Figure size 1200x1200 with 0 Axes>